<a href="https://colab.research.google.com/github/ShawneilRodrigues/deep-learning/blob/main/hyper_parameter_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd

In [19]:
df=pd.read_csv('diabetes.csv')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [21]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [22]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [23]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [24]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [72]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [27]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

In [28]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_split=0.2)

Epoch 1/100
16/16 [==============================] - 4s 105ms/step - loss: 0.7923 - accuracy: 0.4073 - val_loss: 0.8039 - val_accuracy: 0.3577
Epoch 2/100
16/16 [==============================] - 0s 31ms/step - loss: 0.7324 - accuracy: 0.4623 - val_loss: 0.7443 - val_accuracy: 0.4553
Epoch 3/100
16/16 [==============================] - 0s 30ms/step - loss: 0.6835 - accuracy: 0.5642 - val_loss: 0.6957 - val_accuracy: 0.5285
Epoch 4/100
16/16 [==============================] - 0s 19ms/step - loss: 0.6438 - accuracy: 0.6375 - val_loss: 0.6542 - val_accuracy: 0.6341
Epoch 5/100
16/16 [==============================] - 0s 30ms/step - loss: 0.6096 - accuracy: 0.7026 - val_loss: 0.6211 - val_accuracy: 0.6667
Epoch 6/100
16/16 [==============================] - 0s 30ms/step - loss: 0.5831 - accuracy: 0.7291 - val_loss: 0.5927 - val_accuracy: 0.7154
Epoch 7/100
16/16 [==============================] - 0s 23ms/step - loss: 0.5612 - accuracy: 0.7373 - val_loss: 0.5705 - val_accuracy: 0.7398
Epoch

In [30]:
model.evaluate(x_test,y_test)

5/5 [==============================] - 0s 12ms/step - loss: 0.4532 - accuracy: 0.8117


[0.45317158102989197, 0.8116883039474487]

In [31]:
!pip install  keras-tuner # Install the keras-tuner package
import kerastuner as kt # Now import the module in the same cell

In [32]:
def  build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [33]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='project',project_name='diabetes')

In [34]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 2 Complete [00h 00m 03s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7402597665786743
Total elapsed time: 00h 00m 07s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [36]:
model=tuner.get_best_models(num_models=1)[0]

In [37]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.5745 - accuracy: 0.6873 - val_loss: 0.5252 - val_accuracy: 0.7338
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5583 - accuracy: 0.6987 - val_loss: 0.5111 - val_accuracy: 0.7532
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5450 - accuracy: 0.7117 - val_loss: 0.4983 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5334 - accuracy: 0.7150 - val_loss: 0.4864 - val_accuracy: 0.7662
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5223 - accuracy: 0.7313 - val_loss: 0.4766 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5135 - accuracy: 0.7329 - val_loss: 0.4692 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5062 - accuracy: 0.7378 - val_loss: 0.4621 - val_accuracy: 0.7857
Epoch 14

In [52]:
def build_model1(hp):
  model=Sequential()
  units=hp.Int('units',min_value=32,max_value=128)
  model.add(Dense(units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [54]:
tuner1=kt.RandomSearch(build_model1,objective='val_accuracy',max_trials=5,directory='project',project_name='d')

In [55]:
tuner1.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8376623392105103

Best val_accuracy So Far: 0.8376623392105103
Total elapsed time: 00h 00m 12s


In [56]:
tuner1.get_best_hyperparameters()[0].values

{'units': 91}

In [57]:
def build_model2(hp):
  model=Sequential()

  model.add(Dense(91,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(91,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [58]:
tuner2=kt.RandomSearch(build_model2,objective='val_accuracy',max_trials=5,directory='project',project_name='lilo')

In [59]:
tuner2.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 20s


In [60]:
tuner2.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [61]:
model=tuner2.get_best_models(num_models=1)[0]

In [62]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 21ms/step - loss: 0.4555 - accuracy: 0.7704 - val_loss: 0.4563 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4493 - accuracy: 0.7834 - val_loss: 0.4569 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4382 - accuracy: 0.7932 - val_loss: 0.4499 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4310 - accuracy: 0.7932 - val_loss: 0.4613 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4259 - accuracy: 0.8062 - val_loss: 0.4586 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4185 - accuracy: 0.7932 - val_loss: 0.4833 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4124 - accuracy: 0.8094 - val_loss: 0.4748 - val_accuracy: 0.7727
Epoch 14

In [73]:
def build_model3(hp):
  model=Sequential()
  counter=0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                      input_dim=8
                      )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
                      ))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [75]:
tuner3=kt.RandomSearch(build_model3,objective='val_accuracy',max_trials=5,directory='project',project_name='popaha')

In [76]:
tuner3.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 21s


In [77]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 104,
 'activation0': 'relu',
 'dropout0': 0.2,
 'optimizer': 'sgd',
 'units1': 72,
 'activation1': 'tanh',
 'dropout1': 0.5,
 'units2': 40,
 'activation2': 'tanh',
 'dropout2': 0.2,
 'units3': 112,
 'activation3': 'tanh',
 'dropout3': 0.9}

In [78]:
model=tuner3.get_best_models(num_models=1)[0]

In [79]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 20ms/step - loss: 0.5882 - accuracy: 0.7101 - val_loss: 0.5191 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5845 - accuracy: 0.7134 - val_loss: 0.5032 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5609 - accuracy: 0.7199 - val_loss: 0.4948 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5635 - accuracy: 0.7101 - val_loss: 0.4860 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5451 - accuracy: 0.7248 - val_loss: 0.4771 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5632 - accuracy: 0.7248 - val_loss: 0.4714 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5339 - accuracy: 0.7394 - val_loss: 0.4667 - val_accuracy: 0.7922
Epoch 14